In [ ]:
import keras
import numpy as np
import os
import tensorflow as tf
import sys

from keras import backend as K
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
# from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate, Activation, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import optimizers
from keras.models import model_from_json
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from keras.callbacks import ModelCheckpoint
from skimage.transform import resize
from skimage.io import imsave

session = keras.backend.get_session()

print("Python version",sys.version)
print ("keras version", keras.__version__)
print ("tensorflow version", tf.__version__)
print ("numpy version", np.__version__)
# print("Eager mode", tf.executing_eagerly())

In [ ]:
# Get images
X = []
for filename in os.listdir('../data/Train/'):
    X.append(img_to_array(load_img('../data/Train/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X

In [ ]:
tf.compat.v1.disable_eager_execution()
# print("Eager mode", tf.executing_eagerly())
#Load weights
inception = InceptionResNetV2(weights=None, include_top=True)
inception.load_weights('inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
# inception = InceptionResNetV2(weights='imagenet', include_top=True)
inception.graph = tf.compat.v1.get_default_graph()

In [ ]:
# load json and create model
json_file = open('model_colorme.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

optz = tf.optimizers.Adam(lr=0.00001)
#optz = tf.optimizers.rmsprop(lr=0.00001)
model.compile(optimizer=optz, loss='mse', metrics=['acc'])
model.summary()

# load weights into new model
#model.load_weights("model.h5")
model.load_weights("colorme_weight11.h5",by_name=True)
print("Loaded weights from checkpoint")

In [ ]:
filepath="weights-improvement-{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1,save_best_only=True,mode='max')
callbacks_list = [checkpoint]

In [ ]:
#Create embedding
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with session.as_default():
        with inception.graph.as_default():
            embed = inception.predict(grayscaled_rgb_resized)
    return embed

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.4,
        zoom_range=0.4,
        rotation_range=40,
        horizontal_flip=True)

#Generate training data
batch_size = 5

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)

In [ ]:
#Train model
model.fit(image_a_b_gen(batch_size), epochs=100, steps_per_epoch=200, callbacks=callbacks_list, verbose=1)

# Save model
model_json = model.to_json()
with open("model_colorme.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("colorme_weight12.h5")

#Make predictions on validation images
color_me = []
for filename in os.listdir('../data/Test/'):
    color_me.append(img_to_array(load_img('../data/Test/'+filename)))
color_me = np.array(color_me, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
color_me_embed = create_inception_embedding(gray_me)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


# Test model
output = model.predict([color_me, color_me_embed])
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))